## DOCHERTY Ronan et HOUNGUEVOU Thomas
## Rapport réseaux de neurones
### Sujet: Génération de synopsis d'anime
Réseau utilisé: **Transformer**

In [1]:
print("Hello guys this is my top 5 anime list")

Hello guys this is my top 5 anime list
